In [2]:
from selenium import webdriver
import pymongo
import pandas as pd
import requests

In [3]:
# 함수로 만들기
def get_articles(page):
    driver =  webdriver.Chrome()
    url = 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&page={}'.format(page)
    driver.get(url)
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
    
    #제목
    title_list = []
    for article in articles:
        title = article.find_element_by_css_selector("div.assetText > a > h3").text 
        title_list.append(title)
    title =pd.DataFrame(title_list, columns=['title'])
    
    #링크만 뽑아내기
    link_list = []
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        link_list.append(link)
    link= pd.DataFrame(link_list, columns=['link'])
    
    #상세내용 뽑아내기
    preview_list = []
    for article in articles:
        preview = article.find_element_by_css_selector("div.assetText > a > p").text
        preview_list.append(preview)
    preview=pd.DataFrame(preview_list,columns=['preview'])
    
    result=pd.concat([title, link,preview], axis=1)
    driver.quit()
    return result

# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def zd_net(startpage, endpage):
    import pandas as pd
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_articles(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df
final_df= zd_net(1,2)      

In [4]:
final_df.to_csv("csv_files/zd_net.csv", index=False, encoding='utf-8' )


In [5]:
#Techcrunch
def get_crunch(page):
    from selenium import webdriver
    import time
    import pandas as pd
    
    driver =  webdriver.Chrome()
    url = 'http://www.itworld.co.kr/search/?q=AI'
    driver.get(url)
       
    driver.find_element_by_xpath("""//*[@id="___gcse_0"]/div/div/div/div[5]/div[2]/div[1]/div/div[2]/div/div[%s]""" %page).click()
    time.sleep(1)
    
    
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("#___gcse_0 > div > div > div > div.gsc-wrapper > div.gsc-resultsbox-visible > div.gsc-resultsRoot.gsc-tabData.gsc-tabdActive > div > div.gsc-expansionArea > div > div.gs-webResult.gs-result")
    
    
    #제목
    title_list = []
    for article in articles:
        title = article.find_element_by_css_selector("div.gsc-thumbnail-inside > div > a").text 
        title_list.append(title)
    title =pd.DataFrame(title_list, columns=['title'])
    
   
    #링크만 뽑아내기
    link_list = []
    for article in articles:
        link = article.find_element_by_css_selector("div.gsc-url-top > div.gs-bidi-start-align.gs-visibleUrl.gs-visibleUrl-long").text
        link_list.append(link)
    link= pd.DataFrame(link_list, columns=['link'])
    
    #preview 뽑아내기
    preview_list = []
    for article in articles:
        preview = article.find_element_by_css_selector(" div.gsc-table-result > div.gsc-table-cell-snippet-close > div.gs-bidi-start-align.gs-snippet").text


        preview_list.append(preview)
    preview=pd.DataFrame(preview_list,columns=['preview'])
    
    result=pd.concat([title, link, preview], axis=1) #axis=1 은 열 !!! 
   
    driver.quit()
    return result

# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def Tech_crunch(startpage, endpage):
    import pandas as pd
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_crunch(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df

In [6]:
final_df2= Tech_crunch(1,2)
final_df2.to_csv("csv_files/Tech_crunch.csv", index=False, encoding='utf-8' )

In [16]:
import pymongo
import pandas as pd
import requests

# 데이터베이스 서버 연결
client = pymongo.MongoClient("mongodb://localhost:27017/")

#컬렉션 삭제
client.crawling.drop_collection("news_db")

# 데이터 베이스와 컬렉션 생성
#서버.데이터베이스.컬렉션
news_db = client.crawling.news_db


final_df=pd.read_csv("csv_files/zd_net.csv",encoding="utf-8")
final_df2=pd.read_csv("csv_files/Tech_crunch.csv",encoding="utf-8")

items1= final_df.to_dict("records")
items2= final_df2.to_dict("records")

ids=news_db.insert(items1)
ids=news_db.insert(items2)

ConnectionFailure: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다